In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import holodeck.detstats as ds
import h5py

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Calculate Library Detstats

In [ ]:
# hdf_name = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/ss_lib.hdf5'
# detstats_path = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/detstats'

# npsrs = 40
# sigma = 1e-7
# output_dir = detstats_path+'/ds2_psrs40_sigma1e-7'

In [ ]:
# vals = ds.detect_lib(hdf_name, output_dir, npsrs, sigma, 
#                      nskies=25, thresh=0.5, plot=True, debug=True)

# Read in Library Detstats

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
ssfile.close()

print('N,F,R,L:', hc_ss.shape) 
shape = hc_ss.shape
N, F, R, L = shape[0], shape[1], shape[2], shape[3]

# dsfile = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/detstats/dp4_psrs60_sigma1e-07/detstats.npz')
dsfile = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-12-mbp-ss16_n10_r10_f70_d12.5_l10_p0/detstats/ds2_psrs60_sigma5e-07/detstats.npz')
dp_ss = dsfile['dp_ss']
dp_bg = dsfile['dp_bg']
df_ss = dsfile['df_ss']
df_bg = dsfile['df_bg']
snr_ss = dsfile['snr_ss']
snr_bg = dsfile['snr_bg']
print('N,R,S:', dp_ss.shape)
S = dp_ss.shape[-1]

In [ ]:
print('snr_ss range:', np.min(snr_ss), np.max(snr_ss))

# SNR Histogram

In [ ]:
print(snr_ss.shape, snr_bg.shape)
print(snr_ss.min())

In [ ]:
flat_snr = snr_ss.flatten()
bins = np.geomspace(1e-7, flat_snr.max(), 100+1)
plt.hist(flat_snr, bins=bins)

plt.yscale('log')
plt.xscale('log')
plt.title('Histogram of SNRs for the 10th Loudest Sources in each Frequency Bin')
plt.ylabel('Number of Single Sources')
plt.xlabel('SNR_ss')

# Integration Outputs

In [ ]:
from scipy import integrate, special
from sympy import nsolve, Symbol

def _Fe_thresh(Num, alpha_0=0.001):
    """ Calculate the threshold F_e statistic using sympy.nsolve
    
    Parameters
    ----------
    Num : int
        Number of single sources to detect.
    alpha_0 : scalar
        False alarm probability max.

    Returns
    -------
    Fe_bar : scalar
        Threshold Fe statistic
    """
    Fe_bar = Symbol('Fe_bar')
    func = 1 - (1 - (1 + Fe_bar)*np.e**(-Fe_bar))**Num - alpha_0
    Fe_bar = nsolve(func, Fe_bar, 10)
    return(Fe_bar)

def _integrand(Fe, rho):

    I_1 = special.i1(rho*np.sqrt(2*Fe))
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    gamma) : float
    """
    gamma_ssi = integrate.quad(_integrand, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

Find Fe_bar

In [ ]:
Num = F*L
Fe_bar = _Fe_thresh(Num)
outname = './output/integral_maps/Num%d_F%d_L%d_Febar%.2f.dat' % (Num, F, L, Fe_bar)
Fe_bar = np.float64(Fe_bar)

run_integration

In [ ]:
rho = np.geomspace(10**-3, 10**3, 10**3)
Fe = np.linspace(start=15, stop=1000, num=10)

def run_integration(rho, Fe):
    ans = np.zeros_like(rho)
    integrands = np.zeros((len(rho),len(Fe)))
    for ii in range(len(rho)):
        if ii%(len(rho)/10) == 0:
            print(ii, 'out of', len(rho))
        ans[ii] = integrate_rho(Fe_bar, rho[ii])
        for ff in range(len(Fe)):
            integrands[ii,ff] = _integrand(Fe[ff], rho[ii])

    return ans, integrands

ans, integrands = run_integration(rho, Fe)


In [ ]:
rho2 = np.geomspace(10**-1, 10**2, 10**4)
ans2, integrands2 = run_integration(rho2, Fe)


Plot gamma vs. rho

In [ ]:
xx = rho
yy = ans

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:
rho1 = np.geomspace(10**0.5, 10**2, 10**3)
print(rho1.shape)
ans1 = np.zeros_like(rho1)


for ii in range(len(rho1)):
    if ii%(len(rho1)/10) == 0:
       print('on', ii, 'out of', len(rho1))
    ans1[ii] = integrate_rho(Fe_bar, rho1[ii])

In [ ]:
xx=rho1
yy=ans1
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:
print(Fe_bar)
Fe = np.linspace(start=15, stop=1000, num=10)
integrands = np.zeros((len(Fe), len(rho)))
I1 = np.zeros_like(integrands)
for ff in range(len(Fe)):
    if ff%(len(Fe)/10) == 0:
        print('on', ff, 'out of', len(Fe))
        for ii in range(len(rho)):
            integrands[ff,ii] = _integrand(Fe=Fe[ff], rho=rho[ii])
            I1[ff,ii] = special.i1(rho[ii]*np.sqrt(2*Fe[ff]))

plot integrands

In [ ]:

# for ff in range(len(Fe)):
#     plt.loglog(rho, integrands[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
# plt.legend(loc='lower left')
# plt.xlabel(r'$\rho$')
# plt.ylabel('integrand')
# plt.plot(np.arange(len(Fe)), Fe)

xx=rho
yy=integrands
colors = cm.rainbow(np.linspace(0,1,len(Fe)))
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
for ff in range(len(Fe)):
    ax1.loglog(rho, yy[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
    ax2.scatter(xx, np.isnan(yy[ff]), marker='x', s=1, color=colors[ff], alpha=0.5)

# ax1.loglog(xx, yy[0], label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$', 
               fontsize=12)
ax1.set_ylabel('integrand')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='integrand=1')
ax1.legend()


ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:

# for ff in range(len(Fe)):
#     plt.loglog(rho, integrands[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
# plt.legend(loc='lower left')
# plt.xlabel(r'$\rho$')
# plt.ylabel('integrand')
# plt.plot(np.arange(len(Fe)), Fe)

xx=rho
yy=I1
colors = cm.rainbow(np.linspace(0,1,len(Fe)))
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1, 1]})
for ff in range(len(Fe)):
    ax1.loglog(rho, yy[ff]/10**50, color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
    ax2.scatter(xx, np.isnan(yy[ff]), marker='x', s=1, color=colors[ff], alpha=0.5)
    ax3.scatter(xx, np.isinf(yy[ff]), marker='x', s=1, color=colors[ff], alpha=0.5)

# ax1.loglog(xx, yy[0], label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) $', 
               fontsize=12)
ax1.set_ylabel(r'$\mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) /10^{50}$')
# ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.legend()
ax1.set_ylim(10**-54, 10**19) # it won't let me set ymax above 10^19


ax2.set_ylabel('is nan')
ax2.set_ylim(-0.1,1.1)
ax3.set_ylabel('is inf')
ax3.set_ylim(-0.1,1.1)
ax3.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:
print(I1[np.isinf(I1)].size)
print(I1.size)

## Try setting inf I1 to large

In [ ]:

def _integrand_max(Fe, rho):

    I_1 = special.i1(rho*np.sqrt(2*Fe))
    if np.isinf(I_1): I_1 = 10**300
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho_max(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    gamma) : float
    """
    gamma_ssi = integrate.quad(_integrand_max, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

In [ ]:

def run_integration_max(rho, Fe):
    ans = np.zeros_like(rho)
    integrands = np.zeros((len(rho),len(Fe)))
    I1 = np.zeros_like(integrands)    
    for ii in range(len(rho)):
        if ii%(len(rho)/10) == 0:
            print(ii, 'out of', len(rho))
        ans[ii] = integrate_rho_max(Fe_bar, rho[ii])
        for ff in range(len(Fe)):
            integrands[ii,ff] = _integrand_max(Fe[ff], rho[ii])
            I1[ii,ff] = special.i1(rho[ii]*np.sqrt(2*Fe[ff]))
    return ans, integrands, I1

ans_max, integrands_max, I1 = run_integration_max(rho, Fe)
ans2_max, integrands2_max, I12 = run_integration_max(rho2, Fe)


# rint(Fe_bar)
# Fe = np.linspace(start=15, stop=1000, num=10)
# integrands_max = np.zeros((len(Fe), len(rho)))
# I1_max = np.zeros_like(integrands)
# ans_max = np.zeros_like(rho)

# for ii in range(len(rho)):
#     ans_max[ii] = integrate_rho_max(Fe_bar, rho[ii])
#     if ii%(len(rho)/10) == 0:
#         print('on', ii, 'out of', len(rho))
#         for ff in range(len(Fe)):
#             integrands_max[ff,ii] = _integrand(Fe=Fe[ff], rho=rho[ii])
#             I1_max[ff,ii] = special.i1(rho[ii]*np.sqrt(2*Fe[ff]))

Find max gamma

In [ ]:
print('max gamma = gamma[rho=%.2f] = %.10f' % (rho[np.argmax(ans_max)], np.max(ans_max)))
index_one = 0
ii=0
while index_one==0:
    if(ans_max[ii]>=1): index_one = ii
    ii+=1
print('First gamma>=1 = gamma(rho=%.2f) =  %.10f' % (rho[index_one], ans_max[index_one]))

# print(ans_max[index_one], ans_max[index_one+1], ans_max[index_one+2])
# for ii in range(index_one, len(ans_max)-1):
#     if(ans_max[ii]<ans_max[ii+1]):
#         print(ii, 'increasing')
#     elif((ans_max[ii]>ans_max[ii+1])):
#         print(ii,'decreasing')

deriv = np.diff(ans_max)/ans_max[1:]
plt.scatter(rho[1:], deriv>0)
plt.xscale('log')
plt.axvline(rho[index_one], color='k', linestyle='dashed')



Plot gamma v rho

In [ ]:
xx = rho
yy = ans_max

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.axvline(rho[index_one], label=r'$\gamma_i(\rho=%.2f) = 1$' % rho[index_one], color='green', linestyle='--')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)


fig.tight_layout()

## Try setting inf values to 0

In [ ]:
def _integrand_zero(Fe, rho):

    I_1 = special.i1(rho*np.sqrt(2*Fe))
    if np.isinf(I_1): 
        # print('zeroing at rho = %f' % rho)
        return 0
    # print('NOT zeroing at rho = %f' % rho)
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho_zero(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    """
    gamma_ssi = integrate.quad(_integrand_zero, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

run integration

In [ ]:
def run_integration_zero(rho, Fe):
    ans = np.zeros_like(rho)
    integrands = np.zeros((len(rho),len(Fe)))
    for ii in range(len(rho)):
        if ii%(len(rho)/10) == 0:
            print(ii, 'out of', len(rho))
        ans[ii] = integrate_rho_zero(Fe_bar, rho[ii])
        for ff in range(len(Fe)):
            integrands[ii,ff] = _integrand_zero(Fe[ff], rho[ii])
    return ans, integrands

ans_zero, integrands_zero = run_integration_zero(rho, Fe)
ans2_zero, integrands2_zero = run_integration_zero(rho2, Fe)

find max gamma

In [ ]:
print('max gamma = gamma[rho=%.2f] = %.15f' % (rho[np.argmax(ans_zero)], np.max(ans_zero)))
index_zero = 0
ii=0
while index_zero==0:
    if(ans_zero[ii]>=1): index_zero = ii
    ii+=1
print('First gamma>=1 = gamma(rho=%.2f) =  %.15f' % (rho[index_zero], ans_max[index_zero]))

# print(ans_max[index_one], ans_max[index_one+1], ans_max[index_one+2])
# for ii in range(index_one, len(ans_max)-1):
#     if(ans_max[ii]<ans_max[ii+1]):
#         print(ii, 'increasing')
#     elif((ans_max[ii]>ans_max[ii+1])):
#         print(ii,'decreasing')

# deriv = np.diff(ans_zero)/ans_zero[1:]
# plt.scatter(rho[1:], deriv>0)
# plt.xscale('log')
# plt.axvline(rho[index_zero], color='k', linestyle='dashed')



plot gamma

In [ ]:
xx = rho
yy = ans_zero

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='np.inf $ \mathcal{I}_1 = 0$\n', marker='x', markersize=5, linestyle='')
ax1.set_title('Num=%dx%d=%d, Fe_bar=%.2f\n' % (F,L,Num, Fe_bar) + r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.axvline(rho[index_one], label=r'$\gamma_i(\rho=%.2f) = 1$' % rho[index_one], color='green', linestyle='--')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)


fig.tight_layout()

In [ ]:
xx = rho
yy = ans_zero

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})

ax1.loglog(xx, ans_zero, label='np.inf $ \mathcal{I}_1 = 0$', marker='o', markersize=2, 
           linestyle='', color='tab:green', alpha=0.5)
ax1.loglog(xx, ans, label='np.inf $ \mathcal{I}_1 = $np.nan', marker='+', markersize=2, 
           linestyle='', color='tab:orange', alpha=0.5)
ax1.loglog(xx, ans_max, label='np.inf $ \mathcal{I}_1 = 1e300$', marker='x', markersize=2, 
           linestyle='', color='tab:blue', alpha=0.5)

ax1.set_title('Num=%dx%d=%d, Fe_bar=%.2f\n' % (F,L,Num, Fe_bar) + r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.axvline(rho[index_one], label=r'$\gamma_i(\rho=%.2f) = 1$' % rho[index_one], color='green', linestyle='--')
ax1.legend()

ax2.scatter(xx, np.isnan(ans_zero), marker='x', s=1, color='tab:green', alpha=0.5)
ax2.scatter(xx, np.isnan(ans), marker='x', s=1, color='tab:orange', alpha=0.5)
ax2.scatter(xx, np.isnan(ans_max), marker='x', s=1, color='tab:blue', alpha=0.5)
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)


fig.tight_layout()

In [ ]:
xx = rho
yy = ans_zero

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(8,8))

ax1.loglog(xx, ans, label='np.inf $ \mathcal{I}_1 = $np.nan', marker='x', markersize=2, 
           linestyle='', color='tab:orange', alpha=0.5)
ax2.loglog(xx, ans_zero, label='np.inf $ \mathcal{I}_1 = 0$', marker='x', markersize=2, 
           linestyle='', color='tab:green', alpha=0.5)
ax3.loglog(xx, ans_max, label='np.inf $ \mathcal{I}_1 = 1e300$', marker='x', markersize=2, 
           linestyle='', color='tab:blue', alpha=0.5)

ax1.set_title('Num=%dx%d=%d, Fe_bar=%.2f\n' % (F,L,Num, Fe_bar) + r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)

for ax in (ax1, ax2, ax3):
    ax.set_ylabel('$\gamma_i$')
    ax.axhline(1, color='k', alpha=0.5, ls='--')
    ax.axvline(rho[index_one], linestyle='--')


ax3.set_xlabel(r'$\rho$', fontsize=14)

fig.legend()


fig.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, 
                               gridspec_kw={'height_ratios': [2, 1]})
ax1.set_title('Ratio of gamma_i with inf I_1 vals set to 1e300 / set to 0')
ax1.loglog(xx, ans_max/ans_zero, label='maxed / zero-ed', marker='x', markersize=2, 
           linestyle='', color='tab:blue', alpha=0.5)
ax1.set_ylabel('ratio')
ax1.legend()
ax2.plot(xx, ans_max==ans_zero, color='red', label = 'maxed==zeroed')
ax2.set_ylabel('equals')
ax2.legend()
ax2.set_xlabel(r'$\rho')
fig.tight_layout()

plot integrand

In [ ]:

# for ff in range(len(Fe)):
#     plt.loglog(rho, integrands[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
# plt.legend(loc='lower left')
# plt.xlabel(r'$\rho$')
# plt.ylabel('integrand')
# plt.plot(np.arange(len(Fe)), Fe)

xx=rho
y1=integrands
y2=integrands_zero
colors = cm.rainbow(np.linspace(0,1,len(Fe)))
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharex=True, figsize=(12,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
for ff in range(len(Fe)):
    ax1.loglog(xx, y1[:,ff], color=colors[ff], alpha=0.5)
    ax3.scatter(xx, np.isnan(y1[:,ff]), marker='x', s=1, color=colors[ff], alpha=0.5)
    ax2.loglog(xx, y2[:,ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
    ax4.scatter(xx, np.isnan(y2[:,ff]), marker='x', s=1, color=colors[ff], alpha=0.5)

# ax1.loglog(xx, yy[0], label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title('np.inf $ \mathcal{I}_1 =$np.nan \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$', 
               fontsize=12)
ax2.set_title('np.inf $ \mathcal{I}_1 = 0$ \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$', 
               fontsize=12)
ax1.set_ylabel('integrand')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='integrand=1')
fig.legend()


ax3.set_ylabel('is nan')
ax3.set_xlabel(r'$\rho$', fontsize=14)
ax4.set_xlabel(r'$\rho$', fontsize=14)
ax4.sharey(ax3)

fig.tight_layout()

## Try using bessel approximation

In [ ]:
def _mod_bessel_1(x):
    term1 = np.exp(x)/np.sqrt(2*np.pi*x)
    term2 = 1-3/8/x * (1+5/2/8/x * (1 + 21/3/8/x))
    I_1 = term1*term2
    return I_1

def _integrand_aprx(Fe, rho):

    I_1 = _mod_bessel_1(rho*np.sqrt(2*Fe))
    if np.isinf(I_1): 
        # print('zeroing at rho = %f' % rho)
        return 0
    # print('NOT zeroing at rho = %f' % rho)
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho_aprx(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    """
    gamma_ssi = integrate.quad(_integrand_aprx, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

run integration

In [ ]:
def run_integration_aprx(rho, Fe):
    ans = np.zeros_like(rho)
    integrands = np.zeros((len(rho),len(Fe)))
    for ii in range(len(rho)):
        if ii%(len(rho)/10) == 0:
            print(ii, 'out of', len(rho))
        ans[ii] = integrate_rho_aprx(Fe_bar, rho[ii])
        for ff in range(len(Fe)):
            integrands[ii,ff] = _integrand_aprx(Fe[ff], rho[ii])
    return ans, integrands
ans_aprx, integrands_aprx = run_integration_aprx(rho, Fe)
ans2_aprx, integrands2_aprx = run_integration_aprx(rho2, Fe)

Find DP=1

In [ ]:
print(rho.shape, ans_aprx.shape)

In [ ]:
print('max gamma = gamma[rho=%.2f] = %.10f' % (rho[np.argmax(ans_aprx)], np.max(ans_aprx)))
index_aprx = 0
ii=0
while index_aprx==0 and ii<len(ans_aprx):
    if(ans_aprx[ii]>=1): index_aprx = ii
    ii+=1
print('First gamma>=1 = gamma(rho=%.2f) =  %.10f' % (rho[index_aprx], ans_aprx[index_aprx]))
# print(ans_aprx)

In [ ]:
xx = rho
yy = ans_aprx

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='approx $ \mathcal{I}_1$\n', marker='x', markersize=5, linestyle='')
ax1.set_title('Num=%dx%d=%d, Fe_bar=%.2f\n' % (F,L,Num, Fe_bar) + r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.axvline(rho[index_one], label=r'$\gamma_i(\rho=%.2f) = 1$' % rho[index_one], color='green', linestyle='--')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)


fig.tight_layout()

plot integrands

In [ ]:

# for ff in range(len(Fe)):
#     plt.loglog(rho, integrands[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
# plt.legend(loc='lower left')
# plt.xlabel(r'$\rho$')
# plt.ylabel('integrand')
# plt.plot(np.arange(len(Fe)), Fe)

xx=rho
y1=integrands
y2=integrands_aprx
colors = cm.rainbow(np.linspace(0,1,len(Fe)))
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharex=True, figsize=(12,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
for ff in range(len(Fe)):
    ax1.loglog(rho, y1[:,ff], color=colors[ff], alpha=0.5)
    ax3.scatter(xx, np.isnan(y1[:,ff]), marker='x', s=1, color=colors[ff], alpha=0.5)
    ax2.loglog(rho, y2[:,ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
    ax4.scatter(xx, np.isnan(y2[:,ff]), marker='x', s=1, color=colors[ff], alpha=0.5)

# ax1.loglog(xx, yy[0], label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title('np.inf $ \mathcal{I}_1 =$np.nan \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$', 
               fontsize=12)
ax2.set_title('approx $ \mathcal{I}_1$ \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$', 
               fontsize=12)
ax1.set_ylabel('integrand')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='integrand=1')
fig.legend()


ax3.set_ylabel('is nan')
ax3.set_xlabel(r'$\rho$', fontsize=14)
ax4.set_xlabel(r'$\rho$', fontsize=14)
ax4.sharey(ax3)

fig.tight_layout()

In [ ]:
xx = rho
yy = ans_zero

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(6,5))

ax1.loglog(xx, ans, label='np.inf $ \mathcal{I}_1 = $np.nan', marker='x', markersize=2, 
           linestyle='', color='tab:orange', alpha=0.5)
ax2.loglog(xx, ans_aprx, label='aprx $ \mathcal{I}_1$', marker='x', markersize=2, 
           linestyle='', color='tab:green', alpha=0.5)

ax1.set_title('Num=%dx%d=%d, Fe_bar=%.2f\n' % (F,L,Num, Fe_bar) + r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)

for ax in (ax1, ax2):
    ax.set_ylabel('$\gamma_i$')
    ax.axhline(1, color='k', alpha=0.5, ls='--')
    ax.axvline(rho[index_one], linestyle='--')


ax2.set_xlabel(r'$\rho$', fontsize=14)
ax2.sharey(ax1)

fig.legend(loc='lower left')


fig.tight_layout()

## Try conjoined combo

In [ ]:
Fe_bar = np.float64(Fe_bar)
2*np.sqrt(2*Fe_bar)

In [ ]:
def _I1_approx(xx):
    """ Modified Bessel Function of the first kind, first order, expansion for large values.
    """
    term1 = np.exp(xx)/np.sqrt(2*np.pi*xx)
    term2 = 1-3/8/xx * (1+5/2/8/xx * (1 + 21/3/8/xx))
    I_1 = term1*term2
    return I_1

# def _integrand(Fe, rho, cutoff=10.0):
#     xx = rho*np.sqrt(2*Fe)
#     if xx <= cutoff: I_1 = special.i1(xx) 
#     else: I_1 = _I1_approx(xx) # use the approx expansion for large values
#     rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
#     return rv

def _integrand_conj(Fe, rho): 
    xx = rho*np.sqrt(2*Fe)
    I_1 = special.i1(xx) 
    if np.isinf(I_1): # check if nan
        I_1 = _I1_approx(xx) # this could still be inf
        # if np.isinf(I_1): print('I_1(%.2f, %.2f) approx is inf' % (Fe, rho))
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv


def integrate_rho_conj(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    """

    # try high Fe instead of np.inf
    gamma_ssi = integrate.quad(_integrand_conj, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

run integration

In [ ]:

def run_integration_conj(rho, Fe):
    ans = np.zeros_like(rho)
    integrands = np.zeros((len(rho),len(Fe)))
    for ii in range(len(rho)):
        if ii%(len(rho)/10) == 0:
            print(ii, 'out of', len(rho))
        ans[ii] = integrate_rho_conj(Fe_bar, rho[ii])
        for ff in range(len(Fe)):
            integrands[ii,ff] = _integrand_conj(Fe[ff], rho[ii])
    return ans, integrands

ans_conj, integrands_conj = run_integration_conj(rho, Fe)
ans2_conj, integrands2_conj = run_integration_conj(rho2, Fe)

Find DP=1

In [ ]:
print('number of nans:', np.sum(np.isnan(ans_conj)))
# ans_conj[np.isnan(ans_conj)] = 0

print('max gamma = gamma[rho=%.2f] = %.10f' % (rho[np.nanargmax(ans_conj)], np.nanmax(ans_conj)))
index_conj = 0
ii=0
while index_conj==0 and ii<len(ans_conj):
    if(ans_conj[ii]>=1): index_conj = ii
    ii+=1
print('First gamma>=1 = gamma(rho=%.2f) =  %.10f' % (rho[index_conj], ans_aprx[index_conj]))
# print(ans_conj)
print(np.max(ans_conj))

# never quite reaches 1!

In [ ]:
print(ans_conj[np.isinf(ans_conj)]) # none are infinity

plot gamma vs rho

In [ ]:
def plot_gamma(rho, ans, label='', index_one=None):
    xx = rho
    yy = ans

    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                                gridspec_kw={'height_ratios': [3.5, 1]})
    ax1.loglog(xx, yy, label=label, marker='x', markersize=5, linestyle='')
    ax1.set_title('Num=%dx%d=%d, Fe_bar=%.2f\n' % (F,L,Num, Fe_bar) + r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
                fontsize=12)
    ax1.set_ylabel('$\gamma_i$')
    ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
    if index_one is not None:
        ax1.axvline(rho[index_one], label=r'$\gamma_i(\rho=%.2f) = %.2f$' 
                    % (rho[index_one], yy[index_one]), color='green', linestyle='--')
    ax1.legend()

    ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
    ax2.set_ylabel('is nan')
    ax2.set_xlabel(r'$\rho$', fontsize=14)


    fig.tight_layout()
    return fig

fig = plot_gamma(rho, ans_conj, label='Conjoined $ \mathcal{I}_1$', index_one=np.nanargmax(ans_conj))
fig = plot_gamma(rho2, ans2_conj, label='Conjoined $ \mathcal{I}_1$', index_one=np.nanargmax(ans2_conj))

In [ ]:
def plot_3_gamma(rho, ans1, ans2, ans3, label1, label2, label3):
    xx = rho

    fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(6,8))

    ax1.loglog(xx, ans1, label=label1, marker='x', markersize=2, 
            linestyle='', color='tab:orange', alpha=0.5)
    ax2.loglog(xx, ans2, label=label2, marker='x', markersize=2, 
            linestyle='', color='tab:green', alpha=0.5)
    ax3.loglog(xx, ans3, label=label3, marker='x', markersize=2, 
            linestyle='', color='tab:blue', alpha=0.5)

    ax1.set_title('Num=%dx%d=%d, Fe_bar=%.2f\n' % (F,L,Num, Fe_bar) + r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
                fontsize=12)

    label_cutoff = r'cutoff $\rho \sqrt{2 \bar{\mathcal{F}}_e}$ = 0.0'
    for ax in (ax1, ax2, ax3):
        ax.set_ylabel('$\gamma_i$')
        ax.axhline(1, color='k', alpha=0.5, ls='--')
        ax.axvline(rho[index_one], linestyle='--')
        ax.axvline((0.0/np.sqrt(2*Fe_bar)), linestyle='--', color='pink', label=label_cutoff)
        label_cutoff=None



    ax3.set_xlabel(r'$\rho$', fontsize=14)

    fig.legend(loc='lower left')
    fig.tight_layout()
    return fig

fig=plot_3_gamma(rho, ans, ans_aprx, ans_conj, label1='np.inf $ \mathcal{I}_1 = $np.nan', label2='aprx $ \mathcal{I}_1$', label3='conjoined $ \mathcal{I}_1$')

In [ ]:

# for ff in range(len(Fe)):
#     plt.loglog(rho, integrands[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
# plt.legend(loc='lower left')
# plt.xlabel(r'$\rho$')
# plt.ylabel('integrand')
# plt.plot(np.arange(len(Fe)), Fe)

xx=rho
y1=integrands
y2=integrands_conj
colors = cm.rainbow(np.linspace(0,1,len(Fe)))
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharex=True, figsize=(12,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
for ff in range(len(Fe)):
    ax1.loglog(rho, y1[:,ff], color=colors[ff], alpha=0.5)
    ax3.scatter(xx, np.isnan(y1[:,ff]), marker='x', s=1, color=colors[ff], alpha=0.5)
    ax2.loglog(rho, y2[:,ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
    ax4.scatter(xx, np.isnan(y2[:,ff]), marker='x', s=1, color=colors[ff], alpha=0.5)

# ax1.loglog(xx, yy[0], label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title('np.inf $ \mathcal{I}_1 =$np.nan \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$', 
               fontsize=12)
ax2.set_title('conjoined $ \mathcal{I}_1$, cutoff=0 \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$', 
               fontsize=12)
ax1.set_ylabel('integrand')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='integrand=1')
fig.legend()


ax3.set_ylabel('is nan')
ax3.set_xlabel(r'$\rho$', fontsize=14)
ax4.set_xlabel(r'$\rho$', fontsize=14)
ax4.sharey(ax3)

fig.tight_layout()

## Try simplified expansion

In [ ]:
def _I1_approx(xx):
    """ Modified Bessel Function of the first kind, first order, expansion for large values.
    """
    term1 = np.exp(xx)/np.sqrt(2*np.pi*xx)
    term2 = 1-3/8/xx * (1+5/2/8/xx * (1 + 21/3/8/xx))
    I_1 = term1*term2
    return I_1

# def _integrand(Fe, rho, cutoff=10.0):
#     xx = rho*np.sqrt(2*Fe)
#     if xx <= cutoff: I_1 = special.i1(xx) 
#     else: I_1 = _I1_approx(xx) # use the approx expansion for large values
#     rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
#     return rv

def _integrand_approx(Fe, rho):
    xx = rho*np.sqrt(2*Fe)
    termA = np.sqrt( Fe / np.pi / xx )/rho 
    termB = np.exp(xx-Fe-rho**2/2)
    termC = 1-(3/8/xx*(1 + 5/2/8/xx * (1 + 21/3/8/xx)))
    return termA * termB * termC

def _integrand_simp(Fe, rho): 
    xx = rho*np.sqrt(2*Fe)
    I_1 = special.i1(xx) 
    if np.isinf(I_1): # check if nan
        rv = _integrand_approx(Fe,rho)
        if np.isnan(rv):
            print('integrand(%.2f, %.2f) approx is nan' % (Fe, rho))
    else:
        rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv


def integrate_rho_simp(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    """

    # try high Fe instead of np.inf
    gamma_ssi = integrate.quad(_integrand_simp, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

run integration

In [ ]:

def run_integration_simp(rho, Fe):
    ans = np.zeros_like(rho)
    integrands = np.zeros((len(rho),len(Fe)))
    for ii in range(len(rho)):
        if ii%(len(rho)/10) == 0:
            print(ii, 'out of', len(rho))
        ans[ii] = integrate_rho_simp(Fe_bar, rho[ii])
        for ff in range(len(Fe)):
            integrands[ii,ff] = _integrand_simp(Fe[ff], rho[ii])
    return ans, integrands

ans_simp, integrands_simp = run_integration_simp(rho, Fe)
ans2_simp, integrands2_simp = run_integration_simp(rho2, Fe)

run Fe integration

In [ ]:
Fe_Fe = np.geomspace(10**0,10**4, 100)
rho_Fe = np.array([10**-1, 10**0, 10**1, 10**2])
integrands_Fe_simp = np.zeros((len(rho_Fe),len(Fe_Fe)))
for ii in range(len(rho_Fe)):
    for ff in range(len(Fe)):
        integrands_Fe_simp[ii,ff] = _integrand_simp(Fe=Fe_Fe[ff], rho=rho_Fe[ii])


integrands_Fe = np.zeros((len(rho_Fe),len(Fe_Fe)))
for ii in range(len(rho_Fe)):
    for ff in range(len(Fe)):
        integrands_Fe[ii,ff] = _integrand(Fe=Fe_Fe[ff], rho=rho_Fe[ii])

Find DP=1

In [ ]:
print('number of nans:', np.sum(np.isnan(ans_simp)))

print('max gamma = gamma[rho=%.2f] = %.10f' % (rho[np.nanargmax(ans_simp)], np.nanmax(ans_simp)))
index_simp = 0
ii=0
while index_simp==0 and ii<len(rho):
    if(ans_simp[ii]>=1): index_simp = ii
    ii+=1
print('First gamma>=1 = gamma(rho=%.2f) =  %.10f' % (rho[index_simp], ans_aprx[index_simp]))
# print(ans_conj)
print(np.max(ans_simp))

# never quite reaches 1!

Plot gamma vs rho

In [ ]:
def plot_gamma(rho, ans, label='', index_one=None):
    xx = rho
    yy = ans

    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                                gridspec_kw={'height_ratios': [3.5, 1]})
    ax1.loglog(xx, yy, label=label, marker='x', markersize=5, linestyle='')
    ax1.set_title('Num=%dx%d=%d, Fe_bar=%.2f\n' % (F,L,Num, Fe_bar) + r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
                fontsize=12)
    ax1.set_ylabel('$\gamma_i$')
    ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
    if index_one is not None:
        ax1.axvline(rho[index_one], label=r'$\gamma_i(\rho=%.2f) = %.2f$' 
                    % (rho[index_one], yy[index_one]), color='green', linestyle='--')
    ax1.legend()

    ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
    ax2.set_ylabel('is nan')
    ax2.set_xlabel(r'$\rho$', fontsize=14)


    fig.tight_layout()
    return fig

fig = plot_gamma(rho, ans_simp, label='Simplified Approx for $ \mathcal{I}_1$=np.inf', index_one=np.nanargmax(ans_simp))
fig = plot_gamma(rho2, ans2_simp, label='Simplified Approx for $ \mathcal{I}_1$=np.inf', index_one=np.nanargmax(ans2_simp))

plot integrand vs Fe

In [ ]:
def plot_integrand_vs_Fe(rho, Fe, integrands1, integrands2, title1, title2):
    # for ff in range(len(Fe)):
    #     plt.loglog(rho, integrands[ff], color=colors[ff], label = 'Fe=%.2f' % Fe[ff], alpha=0.5)
    # plt.legend(loc='lower left')
    # plt.xlabel(r'$\rho$')
    # plt.ylabel('integrand')
    # plt.plot(np.arange(len(Fe)), Fe)

    xx=Fe
    y1=integrands1
    y2=integrands2
    colors = cm.rainbow(np.linspace(0,1,len(rho)))
    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(nrows=3, ncols=2, sharex=True, figsize=(12,5),
                                gridspec_kw={'height_ratios': [3.5, 1,1]})
    for rr in range(len(rho)):
        ax1.loglog(xx, y1[rr,:], color=colors[rr], alpha=0.5)
        ax3.scatter(xx, np.isnan(y1[rr,:]), marker='x', s=3, color=colors[rr], alpha=0.5)
        ax5.scatter(xx, (y1[rr,:]==0), marker='x', s=3, color=colors[rr], alpha=0.5)

        ax2.loglog(xx, y2[rr,:], color=colors[rr], label = '$rho$='+'%.2f' % rho[rr], alpha=0.5)
        ax4.scatter(xx, np.isnan(y2[rr,:]), marker='x', s=3, color=colors[rr], alpha=0.5)
        ax6.scatter(xx, (y2[rr,:]==0), marker='x', s=3, color=colors[rr], alpha=0.5)

    # ax1.loglog(xx, yy[0], label='Num=%dx%d=%d, Fe_bar=%.2f' % (F,L,Num, Fe_bar), marker='x', markersize=5, linestyle='')
    ax1.set_title(title1, fontsize=12)
    ax2.set_title(title2, fontsize=12)
    ax1.set_ylabel('integrand')
    ax2.sharey(ax1)
    ax1.axhline(1, color='k', alpha=0.5, ls='--', label='integrand=1')
    fig.legend()


    ax3.set_ylabel('is nan')
    ax4.sharey(ax3)
    ax5.set_ylabel('is zero')
    ax6.sharey(ax5)

    ax5.set_xlabel(r'$\mathcal{F}_e$', fontsize=14)
    ax6.set_xlabel(r'$\mathcal{F}_e$', fontsize=14)

    fig.tight_layout()
    return fig

title1 = 'np.inf $ \mathcal{I}_1 =$ np.nan \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$'
title2 = 'np.inf $ \mathcal{I}_1$ =simp apprx \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$'
fig = plot_integrand_vs_Fe(rho_Fe, Fe_Fe, integrands_Fe, integrands_Fe_simp, title1, title2)

In [ ]:
print(integrands_Fe_simp[-1])

## Try cutoff

In [ ]:
def _I1_approx(xx):
    """ Modified Bessel Function of the first kind, first order, expansion for large values.
    """
    term1 = np.exp(xx)/np.sqrt(2*np.pi*xx)
    term2 = 1-3/8/xx * (1+5/2/8/xx * (1 + 21/3/8/xx))
    I_1 = term1*term2
    return I_1

# def _integrand(Fe, rho, cutoff=10.0):
#     xx = rho*np.sqrt(2*Fe)
#     if xx <= cutoff: I_1 = special.i1(xx) 
#     else: I_1 = _I1_approx(xx) # use the approx expansion for large values
#     rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
#     return rv

def _integrand_approx(Fe, rho):
    xx = rho*np.sqrt(2*Fe)
    termA = np.sqrt( Fe / np.pi / xx )/rho 
    termB = np.exp(xx-Fe-rho**2/2)
    termC = 1-(3/8/xx*(1 + 5/2/8/xx * (1 + 21/3/8/xx)))
    return termA * termB * termC

def _integrand_cut(Fe, rho, cutoff=20.0): 
    xx = rho*np.sqrt(2*Fe)
    if xx > cutoff:
        rv = _integrand_approx(Fe,rho)
        if np.isnan(rv):
            print('integrand(%.2f, %.2f) approx is nan' % (Fe, rho))
    else:
        I_1 = special.i1(xx) 
        rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv


def integrate_rho_cut(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    """

    # try high Fe instead of np.inf
    gamma_ssi = integrate.quad(_integrand_cut, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

run integration

In [ ]:
def run_integration_cut(rho, Fe):
    ans = np.zeros_like(rho)
    integrands = np.zeros((len(rho),len(Fe)))
    for ii in range(len(rho)):
        if ii%(len(rho)/10) == 0:
            print(ii, 'out of', len(rho))
        ans[ii] = integrate_rho_cut(Fe_bar, rho[ii])
        for ff in range(len(Fe)):
            integrands[ii,ff] = _integrand_cut(Fe[ff], rho[ii])
    return ans, integrands
ans_cut, integrands_cut = run_integration_cut(rho, Fe)
ans2_cut, integrands2_cut = run_integration_cut(rho2, Fe)

run Fe integration

In [ ]:
integrands_Fe_cut = np.zeros((len(rho_Fe),len(Fe_Fe)))
for ii in range(len(rho_Fe)):
    for ff in range(len(Fe)):
        integrands_Fe_cut[ii,ff] = _integrand_cut(Fe=Fe_Fe[ff], rho=rho_Fe[ii])

Find DP=1

In [ ]:
ans_dp = ans_cut

print('number of nans:', np.sum(np.isnan(ans_dp)))

print('max gamma = gamma[rho=%.2f] = %.10f' % (rho[np.nanargmax(ans_dp)], np.nanmax(ans_dp)))
index = 0
ii=0
while index==0 and ii<len(rho):
    if(ans_dp[ii]>=1): index = ii
    ii+=1
print('First gamma>=1 = gamma(rho=%.2f) =  %.10f' % (rho[index], ans_dp[index]))
# print(ans_conj)
print(np.max(ans_dp))

index_cut = index

# never quite reaches 1!

Plot gamma vs rho

In [ ]:

fig = plot_gamma(rho, ans_cut, label='Simplified if $ xx >$ Cutoff', index_one=np.nanargmax(ans_cut))
fig = plot_gamma(rho2, ans2_cut, label='Simplified if $ xx >$ Cutoff', index_one=np.nanargmax(ans2_cut))

plot integrand vs Fe

In [ ]:
title1 = 'np.inf $ \mathcal{I}_1 =$ np.nan \n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$'
title2 = 'Simplified if $ xx >$ Cutoff\n' + r'integrand = $\frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$'
fig = plot_integrand_vs_Fe(rho_Fe, Fe_Fe, integrands_Fe, integrands_Fe_cut, title1, title2)

## Try equation

In [ ]:
Fe_temp = 10
rho_temp = 50
xx = rho_temp*np.sqrt(2*Fe_temp)
termA = np.sqrt( Fe_temp / np.pi / xx )/rho_temp 
termB = np.exp(xx-Fe_temp-rho_temp**2/2)
termC = 1-(3/8/xx*(1 + 5/2/8/xx * (1 + 21/3/8/xx)))
rv = termA*termB*termC

print(rv)

In [ ]:
Fe_temp = 10
rho_temp = 50
xx = rho_temp*np.sqrt(2*Fe_temp)

rv = ((np.sqrt( Fe_temp / np.pi / xx )/rho_temp ) 
      *(np.exp(xx-Fe_temp-rho_temp**2/2))
      *(1-(3/8/xx*(1 + 5/2/8/xx * (1 + 21/3/8/xx)))))
print(rv)

## Try Max Out


In [ ]:
def _integrand_approx(Fe, rho):
    xx = rho*np.sqrt(2*Fe)
    termA = np.sqrt( Fe / np.pi / xx )/rho 
    termB = np.exp(xx-Fe-rho**2/2)
    termC = 1-(3/8/xx*(1 + 5/2/8/xx * (1 + 21/3/8/xx)))
    return termA * termB * termC

def _integrand(Fe, rho): 
    xx = rho*np.sqrt(2*Fe)
    I_1 = special.i1(xx) 
    if np.isinf(I_1): # check if nan
        rv = _integrand_approx(Fe,rho)
    else:
        rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv


def integrate_rho(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    """

    # try high Fe instead of np.inf
    gamma_ssi = integrate.quad(_integrand_simp, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

def gamma_above_peak(gamma_ssi):
    

# Integrate with Euler Method
and bessel function max patch

F(t, S(t)) = gamma(Fe, integrand(Fe)). For some fixed rho
<!-- 
$\frac{dS(t)}{dt} = \mathrm{integrand}(\mathcal{F}_e) = \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} = F(t, S(t))$

$S(t) = \gamma_i(\rho) = \int \frac{dS(\rho)}{d\rho} d\rho$

For my code:

$F(\rho) = \frac{d\gamma(\rho)}{d\rho} = \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2}$

$ \gamma(\rho_{j+1}) = \gamma(\rho_j) + (\rho_{j+1} - \rho_j) \frac{d\gamma(\rho_j)}{dt}$ -->

$\rho$ = fixed

$\frac{dS(t)}{dt} = \frac{\sqrt{2t}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2t} ) e^{-t - \frac{1}{2}\rho^2}$ given by _integrand_max(t, rho)

$\frac{dS(t)}{dt} = F(t, S(t))$

$s1 = s(t1) = s(0) = \int \frac{dS(t)}{dt} dt $ at $t=0$

In [ ]:
# Define parameters
def f(t, rho):
    return _integrand_max(t, rho)

h = 0.0001 # step size
t = np.arange(0, 1+h, h) # numerical grid
s0 = -1 # initial condition, but not sure what this value should actually be, 
# but maybe it doesn't matter because it's just a constant that would be subtracted in finite integral subtraction


s=np.zeros(len(t))
s[0] = s0
print(s)

rr = int(len(rho)/2) # pick a middleish rho
print(rho[rr])
for i in range(0, len(t)-1):
    s[i+1] = s[i] + h*f(t[i], rho[rr])
gamma1= s[-1] - s[0]

plt.scatter(t, s, marker='x', label='rho=%f, gamma(rho)=%f' % (rho[rr], gamma1), s=3)
plt.legend()
plt.xlabel('t')
plt.ylabel('f(t)')

In [ ]:
# Want integral from Fe_bar to infinity

# Define parameters
def f(t, rho):
    return _integrand_max(t, rho)
print(Fe_bar)
h = 1 # step size
t = np.arange(float(Fe_bar),float(Fe_bar)*10000, h) # numerical grid
s0 = 0 # initial condition, not sure what this value should actually be, 
# but it doesn't matter because it's just a constant subtracted out of definite integrals

s=np.zeros(len(t))
s[0] = s0
# print(s)

rr = int(len(rho)/2) # pick a middleish rho
print(rho[rr])
for i in range(0, len(t)-1):
    s[i+1] = s[i] + h*f(t[i], rho[rr])
gamma1= s[-1] - s[0]

plt.scatter(t, s, marker='x', label='rho=%f, gamma(rho)=%f' % (rho[rr], gamma1), s=3)
plt.legend()
plt.xlabel('t')
plt.ylabel('f(t)')

In [ ]:
# Define parameters
def f(t, rho):
    return _integrand_max(t, rho)
h = 10 # step size
t = np.arange(float(Fe_bar), float(Fe_bar)*100, h) # numerical grid
s0 = 0 # it doesn't matter what this is because it's just subtracted out for definite integrals
s=np.zeros(len(t))

gamma = np.zeros_like(rho)
for rr in range(len(rho)):
    if rr%(len(rho)/10) == 0: print('on', rr, 'out of', len(rho))
    for ii in range(0, len(t)-1):
        s[ii+1] = s[ii] + h*f(t[ii], rho[rr])
    gamma[rr] = s[-1] - s[0]

In [ ]:
h=10
plt.loglog(rho, gamma, marker='x', markersize=1, linestyle='', alpha=0.5,
           label = 'Euler Method with h = %d from $\overline{F}_e$ to 100$\overline{F}_e$' % h)
plt.loglog(rho, ans_max, marker='x', markersize=1, linestyle='',  alpha=0.5,
           label = 'Scipy Integration')
plt.xlabel('rho')
plt.ylabel('gamma')
plt.axhline(1, label='gamma=1', linestyle='--', color='k')
plt.legend(loc='lower left')

In [ ]:

plt.loglog(rho, gamma, marker='x', markersize=3, linestyle='')
plt.xlabel('rho')
plt.ylabel('gamma')
plt.title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$ by Euler Method', 
               fontsize=12)
plt.xlim(10**-3, 10**0)
plt.ylim(10**-1, 10)

In [ ]:
rho3 = np.geomspace(10**-3, 10**1, 100)

# Define parameters
def f(t, rho):
    return _integrand_max(t, rho)

h = 1 # step size
t = np.arange(float(Fe_bar), float(Fe_bar)*1000, h) # numerical grid from Fe_bar to ~inf (or just large)
s0 = 0 # it doesn't matter what this is because it's just subtracted out in finite integral subtraction
s=np.zeros(len(t))

xx = rho3
y1 = np.zeros_like(rho3)
y2 = np.zeros_like(rho3)
for rr in range(len(xx)):
    if rr%(len(xx)/10) == 0: print('on', rr, 'out of', len(xx))
    for ii in range(0, len(t)-1):
        s[ii+1] = s[ii] + h*f(t[ii], xx[rr])
    y1[rr] = s[-1] - s[0]
    y2[rr] = integrate_rho(Fe_bar, xx[rr])
gamma3 = y1
ans3 = y2

In [ ]:

plt.loglog(xx, y1, marker='x', markersize=3, linestyle='', label='Euler Method with h = %.3f from $\overline{F}_e$ to $1000\overline{F}_e$' % h)
plt.loglog(xx, y2, marker='x', markersize=3, linestyle='', label='Scipy Integration')
plt.xlabel('rho')
plt.ylabel('gamma')
plt.legend(loc='upper left')

# quadpy
## without extra pars
Setting rho outside of the function, so the integrand is a function only of Fe.

Bessel function fails??

In [ ]:
import quadpy

In [ ]:
def _integrand_qp(Fe):
        rho_qp=rho2[5]
        I_1 = special.i1(rho_qp*np.sqrt(2*Fe))
        rv = (2*Fe)**(1/2) /rho_qp * I_1 * np.exp(-Fe - rho_qp**2 /2)
        return rv

def integrate_rho_quadpy(Fe_bar):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    rv : float
    """
    gamma_ssi = quadpy.quad(_integrand_qp, Fe_bar, np.inf)[0]
    return gamma_ssi

In [ ]:
rho2 = np.geomspace(10**0.5, 10**2, 10**3)
print(rho1.shape)
ans2 = np.zeros_like(rho1)

for ii in range(len(rho2)):
    if ii%(len(rho2)/10) == 0:
       print('on', ii, 'out of', len(rho2))
    rho_qp=rho2[ii]  
    print(rho_qp)
    ans2[ii] = integrate_rho_quadpy(Fe_bar)

## same way as with integrand.py
This doesn't work

In [ ]:
def _integrand_qp(Fe, rho):

    I_1 = special.i1(rho*np.sqrt(2*Fe))
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho_quadpy(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    rv : float
    """
    gamma_ssi = quadpy.quad(_integrand, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

In [ ]:
rho2 = np.geomspace(10**0.5, 10**2, 10**3)
print(rho1.shape)
ans2 = np.zeros_like(rho1)

for ii in range(len(rho2)):
    if ii%(len(rho2)/10) == 0:
       print('on', ii, 'out of', len(rho2))
    ans2[ii] = integrate_rho_quadpy(Fe_bar, rho2[ii])

In [ ]:
alpha_0 = 0.001
Num = 40000
print(Num)

Fe_test = Symbol('Fe_test')
func = 1 - (1 - (1 + Fe_test)*np.e**(-Fe_test))**Num - alpha_0
Fe_test = nsolve(func, Fe_test, 15) # mod from 
print(Fe_test)